In [1]:

from re import A
from pyrsistent import v

from regex import R
from game1 import OnePlayerGame, GameState, Cards
from IPython.display import clear_output

def print_hand(hand):
    print('\t', hand)
    for card_type, num_cards in enumerate(hand):
        print('\t', Cards(card_type), ":", num_cards, '\t(action number', card_type, ')')

def print_game_state(g):
    print_allowed_actions(g)
    print('currentPlayer:', g.gameState.currentPlayer)
    print('\tcurrentHand:\t\t\t\topposingHand:')
    for card_type, num_cards in enumerate(g.gameState.currentHand):
        print(Cards(card_type), ":\t", num_cards, '\t(action number', card_type, ')\t\t', g.gameState.opposingHand[card_type])
    print('\n')
    deckStr = ""
    for i, card in enumerate(g.gameState.deck):
        deckStr += str(Cards(card))
        if i % 10 == 0:
            deckStr += '\n'

    # print_hand(g.gameState.currentHand)
    # print('opposingHand:')
    # print_hand(g.gameState.opposingHand)

def print_allowed_actions(g):
    print('Allowed actions:')
    for action in g.gameState._allowedActions():
        print('\t', Cards(action), ":",  action)

class EKState:
    def __init__(self, stateStr):
        self.pieces = stateStr.split("|")
        self.currHand = self._getVec(self.pieces[0])
        self.currHand = [int(x) for x in self.currHand]
        self.oppHand = self._getVec(self.pieces[1])
        self.oppHand = [int(x) for x in self.oppHand]
        self.deck = self._getVec(self.pieces[2])
        self.discard = self._getVec(self.pieces[3])
        
    def _getVec(self, vecStr):
        return vecStr.split(",") if vecStr != '' else []

    def handSizes(self):
        pass

    def printAll(self):
        print("deckSize", len(self.deck))
        print("handSizes", len(self.currHand), len(self.oppHand))
        print("currHand", self.currHand)
        print("oppHand", self.oppHand)
        print("discardSize", len(self.discard))

from collections import defaultdict
import random

def def_value():
    return defaultdict(0.0)

class Experience:
    def __init__(self, s, a, r) -> None:
        self.s = s
        self.a = a
        self.r = r

class SARSALambda:
    def __init__(self, g, Q=None, traces=None) -> None:
        self.g = g
        self.action_space = range(g.action_size)
        self.gamma = 0.01
        self.Q = defaultdict(lambda: defaultdict(int)) if Q is None else Q
        self.traces = defaultdict(lambda: defaultdict(int)) if traces is None else traces
        self.visits =  defaultdict(lambda: defaultdict(int))
        self.learning_rate = 0.2
        self.trace_decay_rate = 0.98
        self.last_experience = None
        self.espilon = 0.9

    def update(self, s, a, r):
        if self.last_experience != None:
            s1 = self.last_experience.s
            a1 = self.last_experience.a
            r1 = self.last_experience.r
            self.traces[s1][a1] += 1
            self.visits[s1][a1] += 1
            td_update = r1 + self.gamma * self.Q[s][a] - self.Q[s1][a1]
            for _s in self.traces:
                for _a in self.traces[_s]:
                    self.Q[_s][_a] += self.learning_rate * td_update * self.traces[_s][_a]
                    self.traces[_s][_a] *= self.gamma * self.trace_decay_rate
        self.last_experience = Experience(s, a, r)

    def update1(self, s, a, r):
        if self.last_experience != None:
            s1 = self.last_experience.s
            a1 = self.last_experience.a
            r1 = self.last_experience.r
            self.Q[s1][a1] += r1 + self.learning_rate * (r1 + self.gamma * self.Q[s][a] - self.Q[s1][a1])
        self.last_experience = Experience(s, a, r)

    def greedy_action(self, s):
        if len(self.Q[s]) == 0:
            return self.g.game._randomAction()
            
        allowedActions = self.g.game.game.gameState._allowedActions()
        actions = [(v, a) for a, v in self.Q[s].items()]
        actions.sort(reverse=True)
        for (v, a) in actions:
            if a in allowedActions:
                return a
        else:
            return self.g.game._randomAction()

    def epsilon_greedy_action(self, s):
        self.espilon *= 0.999999
        if len(self.Q[s]) == 0 or random.uniform(0, 1) < self.espilon:
            return self.g.game._randomAction()
        else:
            return self.greedy_action(s)
        


In [20]:
from game1 import ExplodingKittensSingleRandom

def test1(test_fun):
    g = ExplodingKittensSingleRandom()
    done = False
    # clear_output(wait=True)
    while not done:
        input_action = g.game._randomAction()
        # print('Action selected:', input_action)
        (state, reward, done, info) = g.step(input_action)
        # print("state:", state)
        # print("reward:", reward)
    test_fun(state, reward)

def check1(state, reward):
    # print(state, reward)
    ekstate = EKState(state)
    # ekstate.printAll()
    if reward == -1:
        # if lost game, ensure that there are no difuses in agent hand
        assert ekstate.currHand[0] == 0
    elif reward == 1:
        # if won game, ensure that there are no difuses in own hand
        assert ekstate.oppHand[0] == 0
    else:
        # shouldnt have any other reward
        assert 1 == 2

num_ep = 10000
for i in range(num_ep):
    if i % 100 == 0:
        print(i)
    test1(check1)
print("PASSED")

PASSED


In [2]:
from game1 import ExplodingKittensSingleRandom

g = ExplodingKittensSingleRandom()
model = SARSALambda(g)

In [5]:
num_ep = 1000000
for i in range(num_ep):
    if i % (num_ep / 10) == 0:
        print(i)
        print("espilon", model.espilon)
        print("states", len(model.Q))
        qs = list(model.Q.items())
        if qs:
            print("max actions", max(qs, key=lambda x: len(x)))
    (state, reward, done, info) = g.reset()
    while not done:
        action = model.epsilon_greedy_action(state)
        (new_state, reward, done, info) = g.step(action)
        model.update1(state, action, reward)
        state = new_state

saved_model = model
len(model.visits)

0
espilon 0.7651207634899985
states 151487
max actions ('1,0,0,0,0,1,0,1,1,0|1,1,1,0,1,0,0,0,0,0|Cards.EXPLODING_KITTEN,Cards.CAT1,Cards.CAT4,Cards.CAT3,Cards.FAVOR,Cards.CAT2,Cards.CAT2,Cards.DEFUSE|0,0,0,0,0,0,0,0,0,0', defaultdict(<class 'int'>, {10: 0.0}))
100000
espilon 0.4510478438919166
states 609105
max actions ('1,0,0,0,0,1,0,1,1,0|1,1,1,0,1,0,0,0,0,0|Cards.EXPLODING_KITTEN,Cards.CAT1,Cards.CAT4,Cards.CAT3,Cards.FAVOR,Cards.CAT2,Cards.CAT2,Cards.DEFUSE|0,0,0,0,0,0,0,0,0,0', defaultdict(<class 'int'>, {10: 0.0}))
200000
espilon 0.2656955629803086
states 1045084
max actions ('1,0,0,0,0,1,0,1,1,0|1,1,1,0,1,0,0,0,0,0|Cards.EXPLODING_KITTEN,Cards.CAT1,Cards.CAT4,Cards.CAT3,Cards.FAVOR,Cards.CAT2,Cards.CAT2,Cards.DEFUSE|0,0,0,0,0,0,0,0,0,0', defaultdict(<class 'int'>, {10: 0.0}))
300000
espilon 0.1565422417256268
states 1468024
max actions ('1,0,0,0,0,1,0,1,1,0|1,1,1,0,1,0,0,0,0,0|Cards.EXPLODING_KITTEN,Cards.CAT1,Cards.CAT4,Cards.CAT3,Cards.FAVOR,Cards.CAT2,Cards.CAT2,Cards.DEFUSE|

0

In [ ]:
saved_model2 = saved_model

In [7]:
qs = list(model.Q.items())
print(len(qs), qs[0], len(qs[0][1]))
# qs.sort(key=lambda x: max(x[1].values()))
# print("MINs")
# for i in range(10): 
#     print(qs[i])
# print("MAXs")
# for i in range(1, 11): 
#     print(qs[-i])
print("RANDOMs")
for _ in range(10):
    action, value = random.choice(qs)
    print(action, value)

action, value = max(qs, key=lambda x: len(x[1]))
print("MAX")
print(action, value)
print(model.espilon)

4233139 ('1,0,0,0,0,1,0,1,1,0|1,1,1,0,1,0,0,0,0,0|Cards.EXPLODING_KITTEN,Cards.CAT1,Cards.CAT4,Cards.CAT3,Cards.FAVOR,Cards.CAT2,Cards.CAT2,Cards.DEFUSE|0,0,0,0,0,0,0,0,0,0', defaultdict(<class 'int'>, {10: 0.0})) 1
RANDOMs
1,1,1,1,0,0,0,0,1,0|1,0,0,0,0,1,2,0,1,0|Cards.DEFUSE,Cards.SHUFFLE,Cards.CAT3,Cards.EXPLODING_KITTEN,Cards.CAT1,Cards.CAT3|0,0,0,0,0,0,0,0,0,0 defaultdict(<class 'int'>, {2: 0.0})
1,1,0,0,0,0,0,1,1,0|1,0,0,1,0,0,1,0,0,0|Cards.SHUFFLE,Cards.CAT2,Cards.CAT1,Cards.SKIP,Cards.CAT4,Cards.CAT3,Cards.EXPLODING_KITTEN,Cards.CAT1|0,0,0,0,0,0,0,0,0,0 defaultdict(<class 'int'>, {10: 0.0})
1,0,0,0,0,2,0,0,1,0|1,0,0,0,1,0,1,0,0,0|Cards.CAT2,Cards.EXPLODING_KITTEN,Cards.FAVOR,Cards.ATTACK,Cards.SKIP,Cards.CAT4|0,0,0,0,0,0,0,2,0,0 defaultdict(<class 'int'>, {5: 0.0})
1,0,0,1,0,2,0,0,0,0|1,0,1,0,1,0,0,0,0,0|Cards.CAT3,Cards.CAT2,Cards.ATTACK,Cards.EXPLODING_KITTEN,Cards.CAT4,Cards.CAT4,Cards.CAT3,Cards.CAT2|0,0,0,0,0,0,0,0,0,0 defaultdict(<class 'int'>, {5: 0.0})
2,1,0,0,1,0,2,0,0,

In [8]:
from game1 import ExplodingKittensSingleRandom

num_random = 0
num_turns = 0

def greedy_action(model, s, num_random, num_turns):
    num_turns += 1
    if len(model.Q[s]) == 0:
        num_random += 1
        return (model.g.game._randomAction(), num_random, num_turns)
        
    allowedActions = model.g.game.game.gameState._allowedActions()
    actions = [(v, a) for a, v in model.Q[s].items()]
    actions.sort(reverse=True)
    for (v, a) in actions:
        if a in allowedActions:
            return (a, num_random, num_turns)
    else:
        num_random += 1
        return (model.g.game._randomAction(), num_random, num_turns)

g = ExplodingKittensSingleRandom()
saved_model.g = g
num_won = 0
num_bad = 0
num_ep = 100000
for i in range(num_ep):
    if i % 100 == 0:
        print(i)
    (state, reward, done, info) = g.reset()
    while not done:
        (action, num_random, num_turns) = greedy_action(saved_model, state, num_random, num_turns)
        (new_state, reward, done, info) = g.step(action)
        state = new_state

    if reward == 1:
        num_won += 1
    elif reward == 0:
        num_bad += 1

print("num_won", num_won)
print("num_bad", num_bad)
print("num_random", num_random)
print("num_turns", num_turns)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [ ]:
from game1 import ExplodingKittensSingleRandom
g = ExplodingKittensSingleRandom()
done = False
while not done:
    input_action = g.game._randomAction()
    print('Action selected:', input_action)
    (state, reward, done, info) = g.step(input_action)
    print("state:", state)
    print("reward:", reward)

In [2]:
g = OnePlayerGame()
done = False
# clear_output(wait=True)
while not done:
    print_game_state(g.game)
    input_action = int(input())
    print('Action selected:', input_action)
    (next_state, value, done, invalidAction) = g.step(input_action)
    stateStr = next_state.stringify()
    print("STATESTR:", stateStr)

Allowed actions:
	 Cards.NULL : 10
	 Cards.ATTACK : 1
	 Cards.CAT5 : 9
currentPlayer: 1
	currentHand:				opposingHand:
Cards.DEFUSE :	 1 	(action number 0 )		 1
Cards.ATTACK :	 1 	(action number 1 )		 0
Cards.SKIP :	 0 	(action number 2 )		 0
Cards.FAVOR :	 0 	(action number 3 )		 1
Cards.SHUFFLE :	 0 	(action number 4 )		 0
Cards.CAT1 :	 0 	(action number 5 )		 2
Cards.CAT2 :	 0 	(action number 6 )		 0
Cards.CAT3 :	 0 	(action number 7 )		 0
Cards.CAT4 :	 0 	(action number 8 )		 0
Cards.CAT5 :	 2 	(action number 9 )		 0


Action selected: 10
CURRENT PLAYER: 1
STRING STATE: 1,1,0,0,0,0,0,0,0,2|1,0,0,1,0,2,0,0,0,0|Cards.SKIP,Cards.ATTACK,Cards.CAT2,Cards.CAT4,Cards.CAT4,Cards.CAT3,Cards.FAVOR,Cards.CAT2,Cards.CAT2,Cards.CAT1,Cards.CAT5,Cards.SKIP,Cards.ATTACK,Cards.SKIP,Cards.CAT4,Cards.FAVOR,Cards.CAT2,Cards.CAT4,Cards.CAT3,Cards.CAT5,Cards.SHUFFLE,Cards.EXPLODING_KITTEN,Cards.FAVOR,Cards.SKIP,Cards.ATTACK,Cards.CAT3,Cards.CAT1,Cards.CAT3,Cards.SHUFFLE,Cards.SHUFFLE,Cards.SHUFFLE|
NEW 